In [1]:
require 'nn'
require 'hdf5'

# Loading data

In [2]:
-- Train format is (number of Ngrams, Ngram_size + 1) with last
-- col the count of the N_gram of the line

-- Validation format is (number of words to predict, 50 + Ngrams_size -1)
-- where the 50 columns stands for the 50 words possibilities in the prediction,
-- the next col stands for the current context (goal is to predict the Nth word)


myFile = hdf5.open('2-grams.hdf5','r')
data = myFile:all()
train_2 = data['train']
validation_2 = data['valid']
validation_output = data['valid_output']
myFile:close()

myFile = hdf5.open('3-grams.hdf5','r')
data = myFile:all()
train_3 = data['train']
validation_3 = data['valid']
myFile:close()

myFile = hdf5.open('4-grams.hdf5','r')
data = myFile:all()
train_4 = data['train']
validation_4 = data['valid']
myFile:close()


# Maximum Likelihood Estimation

In [4]:
function build_context_count(count_tensor)
    -- Ngram count (depend on w and context)
    -- {'indexN': {index1-...-indexN-1} : count}
    local F_c_w = {}
    -- F_c dict (independent of w, only context based)
    -- {index1-...-indexN-1 : count all words in c}
    local F_c = {}
    -- N_c dict (independent of w, only context based)
    -- {index1-...-indexN-1 : count unique type of words in c}
    local N_c = {}

    local N = count_tensor:size(1)
    local M = count_tensor:size(2)

    for i=1, N do
        indexN = count_tensor[{i,M-1}]
        
        -- build the key index1-...-indexN-1
        indexes = tostring(count_tensor[{i,1}])
        for j=2, M - 2 do
            indexes = indexes .. '-' .. tostring(count_tensor[{i,j}])
        end
        
        -- Filling F_c_w
        -- case context never seen before:
        -- we look for the reduced context (ie index2-...-indexN-1)
        if F_c_w[indexN] == nil then
            F_c_w[indexN] = {[indexes] = count_tensor[{i, M}]}
        else
            if F_c_w[indexN][indexes] == nil then
                F_c_w[indexN][indexes] = count_tensor[{i, M}]
            else
                F_c_w[indexN][indexes] = count_tensor[{i, M}] + F_c_w[indexN][indexes]
            end
        end
        
        -- Updating F_c and F_c
        if F_c[indexes] == nil then
            F_c[indexes] = count_tensor[{i, M}]
            N_c[indexes] = 1
        else
            F_c[indexes] = count_tensor[{i, M}] + F_c[indexes]
            N_c[indexes] = 1 + N_c[indexes]
        end
    end
    
    return F_c_w, F_c, N_c
end

In [5]:
-- Prediction with the MLE (with Laplace smoothing)

function mle_proba(data, F_c_w, alpha)
    local N = data:size(1)
    local M = data:size(2)
    -- Output format: distribution predicted for each N word along the
    -- 50 possibilities
    local distribution = torch.DoubleTensor(N, 50)

    for i=1, N do
        -- build the key index1-...-indexN-1
        indexes = tostring(data[{i,51}])
        for j=52, M do
            indexes = indexes .. '-' .. tostring(data[{i,j}])
        end
        
        -- Look up in the dictionnary for the 50 possible ngrams asked
        for j=1, 50 do
            indexN = data[{i,j}]
            if F_c_w[indexN] == nil or F_c_w[indexN][indexes] == nil then
                distribution[{i,j}] = alpha
            else
                distribution[{i,j}] = F_c_w[indexN][indexes] + alpha
            end
        end
        -- Debug: case where no n-gram were found (only when alpha=0.)
        if distribution:narrow(1,i,1):sum(2)[{1,1}] == 0 then
            -- Select the first one (most common)
            distribution[{i,1}] = 1
        end
    end
    -- normalization (ie we do the MLE given only the 50 possibilities)
    distribution:cdiv(torch.expand(distribution:sum(2), N, 50))
    
    return distribution
end

In [9]:
function perplexity(distribution, true_words)
    -- exp of the average of the cross entropy of the true word for each line
    -- true words (N_words to predict, one hot true value among 50)
    local perp = 0
    local N = true_words:size(1)
    for i = 1,N do
        mm,aa = true_words[i]:max(1)
        perp = perp + math.log(distribution[{i, aa[1]}])
    end
    perp = math.exp(- perp/N)
    return perp
end

In [6]:
F_c_w, F_c, N_c = build_context_count(train_2)
distribution_mle_2 = mle_proba(validation_2, F_c_w, 1)

In [7]:
F_c_w, F_c, N_c = build_context_count(train_3)
distribution_mle_3 = mle_proba(validation_3, F_c_w, 1)

In [8]:
F_c_w, F_c, N_c = build_context_count(train_4)
distribution_mle_4 = mle_proba(validation_4, F_c_w, 1)

In [11]:
-- Results on the validation set
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_2, validation_output))
print('Result on alpha smoothing 3grams', perplexity(distribution_mle_3, validation_output))
print('Result on alpha smoothing 4grams', perplexity(distribution_mle_4, validation_output))

Result on alpha smoothing 2grams	8.8680603647573	


Result on alpha smoothing 3grams	21.513973503713	


Result on alpha smoothing 4grams	29.942969799245	


# Witten Bell Model

In [93]:
-- Witten Bell:
--
-- p_wb(w|c) = (F_c_w + N_c_. * p_wb(w|c'))/(N_c_. + F_c_.)
function distribution_proba_WB(data, alpha)
    local N = data:size(1)
    local M = data:size(2)
    local distribution = torch.DoubleTensor(N, 50)
    
    -- Loading train of the current gram_size
    filename = M - 49 .. '-grams.hdf5'
    myFile = hdf5.open(filename,'r')
    train = myFile:all()['train']
    myFile:close()
    print(filename)
        
    -- Compute the distribution for current gram_size
    F_c_w, F_c, N_c = build_context_count(train)

    -- Witten Bell lambda computation
    local first = torch.zeros(N, 50)
    local second = torch.zeros(N, 50)
    local denom = torch.zeros(N, 50)
    for i=1,N do
        -- Context
        indexes = tostring(data[{i, 51}])
        for j=52, M do
            indexes = indexes .. '-' .. tostring(data[{i, j}])
        end
        for j=1,50 do
            indexN = data[{i,j}]
            -- case data[{i, j}] = <s>
            if F_c_w[indexN] == nil or F_c_w[indexN][indexes] == nil then
                first[{i, j}] = alpha
            else
                first[{i, j}] = F_c_w[indexN][indexes] + alpha
            end
        end  
        -- Debug: case where no n-gram were found
        if first:narrow(1,i,1):sum(2)[{1,1}] == 0 then
            -- Select the first one (most common)
            first[{i,1}] = 1
        end
        
        -- if indexes not in N_c keys, also not in F_c keys
        -- TODO: merge N_c and F_c
        if N_c[indexes] ~= nil then
            second:narrow(1,i,1):add(N_c[indexes])
            denom:narrow(1,i,1):add(N_c[indexes] + F_c[indexes])
        else
            denom:narrow(1,i,1):add(1)
        end
    end
    -- case bigram
    if M == 51 then
        first:cdiv(torch.expand(first:sum(2), N, 50))
        return first
    else
        local next_context = torch.cat(data:narrow(2,1,50), data:narrow(2,52,M - 51), 2)
        local distribution_next_context = distribution_proba_WB(next_context, alpha)
        -- Recurrence
        return first:add(distribution_next_context:cmul(second)):cdiv(denom)
    end
end

In [97]:
d_wB = distribution_proba_WB(validation_4, 1)
print(d_wB:size())

4-grams.hdf5	


3-grams.hdf5	


2-grams.hdf5	


 3370
   50
[torch.LongStorage of size 2]

 3370
   50
[torch.LongStorage of size 2]



 3370
   50
[torch.LongStorage of size 2]



In [98]:
d_wB_no = distribution_proba_WB(validation_4, 0)
print(d_wB:size())

4-grams.hdf5	


3-grams.hdf5	


2-grams.hdf5	


 3370
   50
[torch.LongStorage of size 2]

 3370
   50
[torch.LongStorage of size 2]



 3370
   50
[torch.LongStorage of size 2]



In [99]:
d_wB:narrow(1,1,1)

Columns 1 to 10
 0.0837  0.0837  0.0853  0.0847  0.0837  0.0837  0.0838  0.0837  0.0941  0.0837

Columns 11 to 20
 0.0838  0.0837  0.0837  0.0838  0.0837  0.0837  0.0847  0.0845  0.0843  0.0843

Columns 21 to 30
 0.0837  0.0838  0.0863  0.0840  0.0837  0.0837  0.0837  0.0865  0.0837  0.0848

Columns 31 to 40
 0.0837  0.0843  0.0837  0.0838  0.0852  0.0837  0.0837  0.0838  0.0842  0.0837

Columns 41 to 50
 0.1815  0.0837  0.0837  0.0848  0.0840  0.0861  0.0837  0.0853  0.0842  0.0837
[torch.DoubleTensor of size 1x50]



In [100]:
d_wB_no:narrow(1,1,1)

Columns 1 to 10
 0.0833  0.0000  0.0017  0.0011  0.0000  0.0000  0.0002  0.0000  0.0111  0.0000

Columns 11 to 20
 0.0002  0.0000  0.0000  0.0002  0.0000  0.0000  0.0011  0.0009  0.0007  0.0007

Columns 21 to 30
 0.0000  0.0002  0.0028  0.0004  0.0000  0.0000  0.0000  0.0030  0.0000  0.0012

Columns 31 to 40
 0.0000  0.0007  0.0000  0.0002  0.0016  0.0000  0.0000  0.0002  0.0005  0.0000

Columns 41 to 50
 0.1041  0.0000  0.0000  0.0012  0.0004  0.0026  0.0000  0.0018  0.0005  0.0000
[torch.DoubleTensor of size 1x50]



In [86]:
distribution_3:narrow(1,1,1)

Columns 1 to 10
 0.0000  0.0000  0.0500  0.0000  0.0000  0.0000  0.0000  0.0000  0.0500  0.0000

Columns 11 to 20
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0500  0.0000

Columns 21 to 30
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.1000  0.0000  0.0500

Columns 31 to 40
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

Columns 41 to 50
 0.6500  0.0000  0.0000  0.0000  0.0000  0.0500  0.0000  0.0000  0.0000  0.0000
[torch.DoubleTensor of size 1x50]



In [87]:
distribution:narrow(1,1,1)

Columns 1 to 10
 0.0000  0.0000  0.0117  0.0078  0.0000  0.0000  0.0013  0.0000  0.0805  0.0000

Columns 11 to 20
 0.0013  0.0000  0.0000  0.0013  0.0000  0.0000  0.0078  0.0065  0.0039  0.0052

Columns 21 to 30
 0.0000  0.0013  0.0208  0.0026  0.0000  0.0000  0.0000  0.0195  0.0000  0.0078

Columns 31 to 40
 0.0000  0.0052  0.0000  0.0013  0.0117  0.0000  0.0000  0.0013  0.0039  0.0000

Columns 41 to 50
 0.7506  0.0000  0.0000  0.0091  0.0026  0.0182  0.0000  0.0130  0.0039  0.0000
[torch.DoubleTensor of size 1x50]



# Accuracy

In [17]:
m, true_output = validation_output:max(2)

In [21]:
function compute_accuracy(pred, true_output)
    max,argmax = pred:max(2)
    acc = 0
    for i = 1, true_output:size(1) do
        if argmax[i][1] == true_output[i][1] then
            acc = acc + 1
        end
    end
    score = acc/true_output:size(1)
    
    return score
end

In [101]:
print('Result on alpha smoothing ', compute_accuracy(distribution, true_output))
print('Result on alpha smoothing ', compute_accuracy(distribution_3, true_output))
print('Result on alpha smoothing ', compute_accuracy(distribution_4, true_output))
print('Result on Witten Bell ', compute_accuracy(d_wB, true_output))
print('Result on Witten Bell ', compute_accuracy(d_wB_no, true_output))


Result on alpha smoothing 	

0.59258160237389	


Result on alpha smoothing 	0.37952522255193	


Result on alpha smoothing 	0.23412462908012	


Result on Witten Bell 	0.32640949554896	


Result on Witten Bell 	0.25786350148368	


# Perplexity

In [125]:
print('Result on alpha smoothing ', perplexity(distribution))
print('Result on alpha smoothing ', perplexity(distribution_3))
print('Result on alpha smoothing ', perplexity(distribution_4))
print('Result on Witten Bell ', perplexity(d_wB))
print('Result on Witten Bell ', perplexity(d_wB_no))

Result on alpha smoothing 	110.46837660031	


Result on alpha smoothing 	64.899935042845	


Result on alpha smoothing 	59.51012776219	


Result on Witten Bell 	5.52172116585	


Result on Witten Bell 	inf	
